In [1]:
from dms_stan.flip_dsets import trpb_instance_factory

In [2]:
liba = trpb_instance_factory(
    "/home/bwittmann/GitRepos/DMSStan/flip3_data/counts/trpb/libA.csv",
    "LibA",
    "exponential",
    "gamma",
)

ValueError: Shape is not broadcastable to parent shapes

In [6]:
liba.__class__.n_variants

AttributeError: type object 'LibA' has no attribute 'n_variants'

In [1]:
from dms_stan.model.pytorch.mle import MLEInferenceRes
import numpy as np
import pandas as pd

from dms_stan.flip_dsets import load_trpb_dataset

In [2]:
for dset in [
    "libA",
    "libB",
    "libC",
    "libD",
    "libE",
    "libF",
    "libG",
    "libH",
    "libI",
    "four-site",
]:
    print(f"Loading dataset {dset}")
    test = load_trpb_dataset(f"/home/bwittmann/GitRepos/DMSStan/flip3_data/counts/trpb/{dset}.csv")
    print(test["starting_counts"].shape)

# test = load_trpb_dataset("/home/bwittmann/GitRepos/DMSStan/flip3_data/counts/trpb/libI.csv")

Loading dataset libA
(9254,)
Loading dataset libB
(9261,)
Loading dataset libC
(9260,)
Loading dataset libD
(9057,)
Loading dataset libE
(9229,)
Loading dataset libF
(8996,)
Loading dataset libG
(9041,)
Loading dataset libH
(8970,)
Loading dataset libI
(9121,)
Loading dataset four-site
(2, 1, 194361)


In [6]:
np.log(10)

np.float64(2.302585092994046)

In [13]:
np.log(test["timepoint_od"] / test["starting_od"][:, None])

array([[2.02814825, 3.0155349 , 3.91999118, 4.09434456, 4.20469262,
        4.24849524],
       [2.02814825, 3.03495299, 3.98154907, 4.17438727, 4.24849524,
        4.31748811]])

In [3]:
pd.read_csv("/home/bwittmann/GitRepos/DMSStan/tmp/trpb/libI/trpb_libI_gamma_exponential_loss-curve.csv")

,-log pdf/pmf,iteration,shifted log(-log pdf/pmf)
0,65159120.0,0,6.382415e+07
1,65064850.0,1,6.372988e+07
2,64970690.0,2,6.363572e+07
3,64876636.0,3,6.354167e+07
4,64782700.0,4,6.344773e+07
...,...,...,...
29584,1334967.0,29584,1.125000e+00
29585,1334966.9,29585,1.000000e+00
29586,1334966.9,29586,1.000000e+00
29587,1334966.9,29587,1.000000e+00


In [4]:
res = MLEInferenceRes.from_disk("/home/bwittmann/GitRepos/DMSStan/tmp/trpb/libI/trpb_libI_gamma_exponential_samples.nc")

In [5]:
res.run_ppc(logy_ppc_samples=True)

BokehModel(combine_events=True, render_bundle={'docs_json': {'3adda285-0345-4553-a741-572ffae6553f': {'version…

In [3]:
variants = data.pop("variants")

In [4]:
model = MODEL_MAP["trpb"]["libI"]["lomax"]["exponential"](**data)

In [5]:
model.prior_predictive()

BokehModel(combine_events=True, render_bundle={'docs_json': {'49336f4d-e2bd-4c0c-8dee-d36ed83f4f4d': {'version…

In [6]:
res = SampleResults.from_disk(
    "/home/bwittmann/GitRepos/DMSStan/tmp/trpb/libI/trpb_libI_exponential_exponential_diagnosed.nc",
    skip_fit=True,
)

In [7]:
res.inference_obj.variable_diagnostic_stats.sel(metric="r_hat").max()

<xarray.Dataset> Size: 84B
Dimensions:               ()
Coordinates:
    metric                <U9 36B 'r_hat'
Data variables:
    log_theta_t0          float64 8B 1.0
    log_r                 float64 8B 1.0
    log_foldchange_sigma  float64 8B 1.0
    r                     float64 8B 1.01
    raw_abundances_tg0    float64 8B 1.01
    log_theta_tg0         float64 8B 1.0

In [8]:
tests = res.inference_obj.variable_diagnostic_tests.log_theta_tg0.sel(metric="r_hat").values
failures = np.nonzero(tests)

In [9]:
worst = res.inference_obj.variable_diagnostic_stats.log_theta_tg0.sel(metric="r_hat").values[failures]

In [10]:
res.inference_obj.posterior.log_foldchange_sigma.values

array([[0.0538241, 0.0522458, 0.0527443, ..., 0.0482759, 0.0481388,
        0.0501887],
       [0.0485601, 0.0481094, 0.0504713, ..., 0.051811 , 0.0530787,
        0.0516946],
       [0.0480358, 0.0508975, 0.0487617, ..., 0.0522703, 0.0518892,
        0.0522187],
       ...,
       [0.0510638, 0.0515819, 0.052384 , ..., 0.0532981, 0.052899 ,
        0.0542892],
       [0.0528009, 0.0544239, 0.0545912, ..., 0.0517875, 0.0539521,
        0.0530968],
       [0.051837 , 0.0535583, 0.0530494, ..., 0.0488985, 0.0485297,
        0.0480919]], shape=(8, 2000), dtype=float32)

In [11]:
res.plot_variable_failure_quantile_traces()

BokehModel(combine_events=True, render_bundle={'docs_json': {'a60ce0df-6037-482d-b12c-2d4c7cce601b': {'version…

In [15]:
res.inference_obj.posterior.sizes["chain"]

8